# Chap.02 クローラーを設計する

## 01 クローラーの設計の基本

### クローラーの種類

### 目的と対象を明確にする

### URL構造を確認する

- サイトマップをツリー構造で提供しているサイト
- サイトマップをXMLで提供しているサイト
- サイトマップが得られない場合

### 目的とするデータの提供がないか確認する

- [Wikipedia:データベースダウンロード](https://ja.wikipedia.org/wiki/Wikipedia:%E3%83%87%E3%83%BC%E3%82%BF%E3%83%99%E3%83%BC%E3%82%B9%E3%83%80%E3%82%A6%E3%83%B3%E3%83%AD%E3%83%BC%E3%83%89)
- [Index of /jawiki/](https://dumps.wikimedia.org/jawiki/)

### Web API

- [Qiita Developer APIv2](https://qiita.com/api/v2/docs)
- [Qiita: Web API アクセスサンプル](https://qiita.com/api/v2/items?page=1&per_page=20)
- [Qiita: Pythonタグが付けられた新着投稿](https://qiita.com/tags/python/feed.atom)

## 02 クローラーの持つ各処理工程ごとの設計と注意点

### 設計の勘所

クローラーの各処理工程

- 収集
  - ネットワーク上のデータにリクエストする。
  - リンクがあれば収集し、さらにそれらのリンクに対してもリクエストする。
- 解析
  - パース。
  - テキストデータを構造化データに変換する。
- 抽出
  - 対象データを構造化データの特定のキーから取り出したり、
  - スクレイピングや正規表現を使って抽出する。
- 加工
  - 利用形態に合わせて、抽出したデータからノイズを取り除いたり、
  - 正規化したり、
  - 画像処理をしたりする。
- 保存
  - 収集したページや、抽出・加工したデータをデータストレージに保存する。

### ネットワークリクエスト

- 間隔を設ける
- タイムアウト
- リトライ

&nbsp;

- Exponential Backoff
  - リトライまでの最適な時間をシステムが計算すること。

### パース（解析）

- 文字コード
  - フォールバック
    - システム障害時に最低限の仕様でシステムを維持すること。
- HTML/XML解析
  - [HTML Tidy Legacy Website](https://tidy.sourceforge.net/)
  - [Beautiful Soup](https://www.crummy.com/software/BeautifulSoup/)
  - [lxml](https://lxml.de/index.html)
- JSONデコーダー

### スクレイピングと正規表現

- URL正規化
  - プロトコル、ホスト名を含んだ絶対リンクに修正する。
- テスト
  - `スクレイピング関数（入力HTML） = 抽出結果として期待する株価の数値`

### データストレージの選定と構造

- ファイル
  - SHA-1形式
    - Secure Hash Algorithm
    - 同規格の暗号を用いたハッシュ関数の形式の1つ
- データベース
  - カラム型BLOB
- データの存在確認
  - URLとは別にURLのハッシュの別のカラムに保存する。

## 03 バッチ作成の注意点

### バッチ（バッチプログラム）とは

- あらかじめ定められた一連の処理の流れを一度に行うこと。

&nbsp;

- 工程ごとの分離
  - ネットワークリクエスト処理
  - スクレイピング処理
  - 分離の粒度のバランス

&nbsp;

- 中間データを保存しておく
  - ネットワークリクエストしたデータの保存
  - リクエストに失敗したページのみをリクエストする仕組み

&nbsp;

- 実行時間と経過がわかるようにする
  - 処理中のURLや工程のラベル表示
  - HTTPステータスコード

&nbsp;

- 停止条件を明確にする
  - 無限ループの回避

&nbsp;

- 関数の引数をシンプルにする
  - 中間データを保存しておき、そのキーを引数にする。

&nbsp;

- 日時の扱いに注意する
  - UTCで統一する。
  - NTPも保存する。

### 設計の例

- [吉川英治の作品リスト](https://www.aozora.gr.jp/index_pages/person1562.html)
  - ソースを確認する
  - 新着の検知の仕組みを考える
  - データの保存方法
  - ファイルの保存形式

&nbsp;

- URLのデータベースインデックスの作成
  - ハッシュ関数（BLAKE2）